## **Fine-tuning BERT for named-entity recognition**

In this notebook, we are going to use **BertForTokenClassification** which is included in the [Transformers library](https://github.com/huggingface/transformers) by HuggingFace. This model has BERT as its base architecture, with a token classification head on top, allowing it to make predictions at the token level, rather than the sequence level. Named entity recognition is typically treated as a token classification problem, so that's what we are going to use it for.

This tutorial uses the idea of **transfer learning**, i.e. first pretraining a large neural network in an unsupervised way, and then fine-tuning that neural network on a task of interest. In this case, BERT is a neural network pretrained on 2 tasks: masked language modeling and next sentence prediction. Now, we are going to fine-tune this network on a NER dataset. Fine-tuning is supervised learning, so this means we will need a labeled dataset.

If you want to know more about BERT, I suggest the following resources:
* the original [paper](https://arxiv.org/abs/1810.04805)
* Jay Allamar's [blog post](http://jalammar.github.io/illustrated-bert/) as well as his [tutorial](http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)
* Chris Mccormick's [Youtube channel](https://www.youtube.com/channel/UCoRX98PLOsaN8PtekB9kWrw)
* Abbishek Kumar Mishra's [Youtube channel](https://www.youtube.com/user/abhisheksvnit)

The following notebook largely follows the same structure as the tutorials by Abhishek Kumar Mishra. For his tutorials on the Transformers library, see his [Github repository](https://github.com/abhimishra91/transformers-tutorials).

NOTE: this notebook assumes basic knowledge about deep learning, BERT, and native PyTorch. If you want to learn more Python, deep learning and PyTorch, I highly recommend cs231n by Stanford University and the FastAI course by Jeremy Howard et al. Both are freely available on the web.  

Now, let's move on to the real stuff!

#### **Importing Python Libraries and preparing the environment**

This notebook assumes that you have the following libraries installed:
* pandas
* numpy
* sklearn
* pytorch
* transformers
* seqeval

As we are running this in Google Colab, the only libraries we need to additionally install are transformers and seqeval (GPU version):

In [2]:
!pip install transformers seqeval[gpu]

     |████████████████████████████████| 3.4 MB 16.0 MB/s 
     |████████████████████████████████| 43 kB 984 kB/s 
     |████████████████████████████████| 61 kB 595 kB/s 
     |████████████████████████████████| 3.3 MB 56.8 MB/s 
     |████████████████████████████████| 895 kB 72.2 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=cc938fe4fa37de6837291057f4034402e4e2065f29cb183b6fd4d84978ca6d66
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/NER_model/Architectures')



Mounted at /content/drive


As deep learning can be accellerated a lot using a GPU instead of a CPU, make sure you can run this notebook in a GPU runtime (which Google Colab provides for free! - check "Runtime" - "Change runtime type" - and set the hardware accelerator to "GPU").

We can set the default device to GPU using the following code (if it prints "cuda", it means the GPU has been recognized):

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)


cuda


#### **Downloading and preprocessing the data**
Named entity recognition (NER) uses a specific annotation scheme, which is defined (at least for European languages) at the *word* level. An annotation scheme that is widely used is called **[IOB-tagging](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging))**, which stands for Inside-Outside-Beginning. Each tag indicates whether the corresponding word is *inside*, *outside* or at the *beginning* of a specific named entity. The reason this is used is because named entities usually comprise more than 1 word. 

Let's have a look at an example. If you have a sentence like "Barack Obama was born in Hawaï", then the corresponding tags would be   [B-PERS, I-PERS, O, O, O, B-GEO]. B-PERS means that the word "Barack" is the beginning of a person, I-PERS means that the word "Obama" is inside a person, "O" means that the word "was" is outside a named entity, and so on. So one typically has as many tags as there are words in a sentence.

So if you want to train a deep learning model for NER, it requires that you have your data in this IOB format (or similar formats such as [BILOU](https://stackoverflow.com/questions/17116446/what-do-the-bilou-tags-mean-in-named-entity-recognition)). There exist many annotation tools which let you create these kind of annotations automatically (such as Spacy's [Prodigy](https://prodi.gy/), [Tagtog](https://docs.tagtog.net/) or [Doccano](https://github.com/doccano/doccano)). You can also use Spacy's [biluo_tags_from_offsets](https://spacy.io/api/goldparse#biluo_tags_from_offsets) function to convert annotations at the character level to IOB format.

Here, we will use a NER dataset from [Kaggle](https://www.kaggle.com/namanj27/ner-dataset) that is already in IOB format. One has to go to this web page, download the dataset, unzip it, and upload the csv file to this notebook. Let's print out the first few rows of this csv file:

Let's check how many sentences and words (and corresponding tags) there are in this dataset:

As we can see, there are approximately 48,000 sentences in the dataset, comprising more than 1 million words and tags (quite huge!). This corresponds to approximately 20 words per sentence. 

Let's have a look at the different NER tags, and their frequency: 

There are 8 category tags, each with a "beginning" and "inside" variant, and the "outside" tag. It is not really clear what these tags mean - "geo" probably stands for geographical entity, "gpe" for geopolitical entity, and so on. They do not seem to correspond with what the publisher says on Kaggle. Some tags seem to be underrepresented. Let's print them by frequency (highest to lowest): 

Let's remove "art", "eve" and "nat" named entities, as performance on them will probably be not comparable to the other named entities. 

Now, we have to ask ourself the question: what is a training example in the case of NER, which is provided in a single forward pass? A training example is typically a **sentence**, with corresponding IOB tags. Let's group the words and corresponding tags by sentence:

Let's have a look at the different NER tags. 

We create 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook.

In [5]:
import nltk
nltk.download('punkt')
labels_to_ids = {'O': 0, 'Resource': 1, 'Device': 2, 'Service': 3}
ids_to_labels = {0: 'O', 1: 'Resource', 2: 'Device', 3: 'Service'}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def judge_entity(w, sentence, index1, index2, entity):
    return entity if w in sentence[index1: index2].split(" ") else 'O'

In [7]:
raw_train = np.load('/content/drive/MyDrive/ColabNotebooks/NER_model/Architectures/TRAIN_DATA_np.npy', allow_pickle=True)
raw_test = np.load('/content/drive/MyDrive/ColabNotebooks/NER_model/Architectures/TEST_DATA_np.npy', allow_pickle=True)
# raw_train = np.load('./TRAIN_DATA_np.npy', allow_pickle=True)
# raw_test = np.load('./TEST_DATA_np.npy', allow_pickle=True)

In [8]:
df_train = []
for instance in raw_train:
    field1 = instance[0]
    field2 = instance[1]['entities'][0][2]
    field4 = list(map(lambda x: judge_entity(x, field1, instance[1]['entities'][0][0], instance[1]['entities'][0][1], field2), 
                      nltk.word_tokenize(field1)))
    df_train.append([field1, field2, ','.join(field4)])
df_train = pd.DataFrame(df_train, columns=['sentence', 'entity', 'word_labels'])
print(df_train.head())

df_test = []
for instance in raw_test:
    field1 = instance[0]
    field2 = instance[1][0][2]
    field4 = list(map(lambda x: judge_entity(x, field1, instance[1][0][0], instance[1][0][1], field2),
                      nltk.word_tokenize(field1)))
    df_test.append([field1, field2, ','.join(field4)])
df_test = pd.DataFrame(df_test, columns=['sentence', 'entity', 'word_labels'])
print(df_test.head())


                                            sentence  ...                                        word_labels
0  It lays a certain theoretical foundation for g...  ...  O,O,O,O,O,O,O,O,O,Device,Device,O,O,O,O,O,O,O,...
1  He's the repository of our common history, and...  ...     O,O,O,Resource,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
2  Supply of ball valves of nominal diameter DN 7...  ...  O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Device,D...
3  Often the meter seems to fluctuate between tri...  ...     O,O,Device,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4  It bans the private sale of all firearms, crea...  ...       O,O,O,O,O,O,O,O,O,O,O,O,O,Resource,O,O,O,O,O

[5 rows x 3 columns]
                                            sentence  ...                                        word_labels
0  Don't take our word for it, check the Water Su...  ...         O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Resource,O
1  The new SQLite allows us to search for specifi...  ...                   O,O,Resource,O,O,O,O,O,O,O,O,O

In [9]:
print(df_train.shape)
print(df_test.shape)
data = pd.concat([df_train, df_test])
data.shape

(4807, 3)
(2589, 3)


(7396, 3)

As we can see, there are now only 10 different tags. 

Let's only keep the "sentence" and "word_labels" columns, and drop duplicates:

In [10]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
print(data.head())
print(data.shape)


                                            sentence                                        word_labels
0  It lays a certain theoretical foundation for g...  O,O,O,O,O,O,O,O,O,Device,Device,O,O,O,O,O,O,O,...
1  He's the repository of our common history, and...     O,O,O,Resource,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
2  Supply of ball valves of nominal diameter DN 7...  O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Device,D...
3  Often the meter seems to fluctuate between tri...     O,O,Device,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4  It bans the private sale of all firearms, crea...       O,O,O,O,O,O,O,O,O,O,O,O,O,Resource,O,O,O,O,O
(7392, 2)


In [11]:
len(data)

7392

Let's verify that a random sentence and its corresponding tags are correct:

In [12]:
data.iloc[41].sentence

'The relay actuator operates at 18-32 volts dc with a maximum current draw of 8 amperes.\tCopy'

In [13]:
data.iloc[41].word_labels

'O,Device,Device,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O'

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [14]:
# MAX_LEN = 128
MAX_LEN = 50 #115
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

A tricky part of NER with BERT is that BERT relies on **wordpiece tokenization**, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level! 

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then we will have to propagate the word’s original label to all of its wordpieces: "b-gpe", "b-gpe", "b-gpe". The model should be able to produce the correct labels for each individual wordpiece. The function below (taken from [here](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L118)) implements this.






In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

Note that this is a **design decision**. You could also decide to only label the first wordpiece of each word and let the model only learn this (this is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063)). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word.

All of them lead to good performance.

Next, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. 

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html).  

In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]  
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [labels_to_ids[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [ ]:
train_size = 0.65
# train_dataset = data.sample(frac=train_size,random_state=200)
# test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
# train_dataset = train_dataset.reset_index(drop=True)
train_dataset = data[:4803]
test_dataset = data[4803:]
train_dataset = train_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (7392, 2)
TRAIN Dataset: (4803, 2)
TEST Dataset: (2589, 2)


Let's have a look at the first training example:

In [ ]:
training_set[0]

{'ids': tensor([  101,  2009, 19764,  1037,  3056,  9373,  3192,  2005,  5016, 16853,
         14122,  7277,  6024,  8846,  2552,  6692,  4263,  2108,  2109,  1999,
          1996,  4249,  1997,  6276,  2007,  1999, 10755, 19210,  2791,  6276,
          2486,  1012,  6100,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'targets': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0])}

Let's verify that the input ids and corresponding targets are correct:

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"]), training_set[0]["targets"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       0
it          0
lays        0
a           0
certain     0
theoretical  0
foundation  0
for         0
giant       0
magnet      0
##ost       0
##ric       0
##tive      0
relay       2
act         2
##ua        2
##tor       2
being       0
used        0
in          0
the         0
fields      0
of          0
cutting     0
with        0
in          0
##var       0
##iable     0
##ness      0
cutting     0
force       0
.           0
copy        0
[SEP]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0


Now, let's define the corresponding PyTorch dataloaders:

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

#### **Training the model**

Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful [cs231n course](http://cs231n.stanford.edu/) at Stanford (see also his [blog post about debugging neural networks](http://karpathy.github.io/2019/04/25/recipe/)). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83. 

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (which is used by `BertForTokenClassification`) uses *mean reduction* by default, it will compute the mean loss for each of the tokens in the sequence (in other words, for all of the 512 tokens). The mean of 512 times -log(1/17) is, you guessed it, -log(1/17).  

Let's verify this:



In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.4075, device='cuda:0', grad_fn=<NllLossBackward0>)

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 50, 4])

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is [included](https://huggingface.co/transformers/main_classes/optimizer_schedules.html) in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        # if idx > 102:
        #    break
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        # loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=targets)
        tmp = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = tmp[0], tmp[1]
        # print(tmp[1])
        # print(tmp[0].item())
        # print(type(loss))
        # print(type(tr_logits))
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print("Training loss per 100 training steps: ", idx, loss_step)
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps:  0 1.2654057741165161
Training loss per 100 training steps:  100 0.4147267853564555
Training loss per 100 training steps:  200 0.32914160551568167
Training loss per 100 training steps:  300 0.28344188944940946
Training loss per 100 training steps:  400 0.260215513064454
Training loss per 100 training steps:  500 0.2410282685891716
Training loss per 100 training steps:  600 0.22559692140377302
Training loss per 100 training steps:  700 0.21382839548881147
Training loss per 100 training steps:  800 0.2025482202653358
Training loss per 100 training steps:  900 0.19374208909756072
Training loss per 100 training steps:  1000 0.18776792490279878
Training loss per 100 training steps:  1100 0.1809434322631841
Training loss per 100 training steps:  1200 0.17474679187324818
Training loss epoch: 0.17474679187324818
Training accuracy epoch: 0.9469156318140086
Training epoch: 2
Training loss per 100 training steps:  0 0.17594194412231445
Train

#### **Evaluating the model**

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            # if idx > 101:
            #     break
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            tmp = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = tmp[0], tmp[1]
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 7.741596346022561e-05
Validation loss per 100 evaluation steps: 0.06130927857837846
Validation loss per 100 evaluation steps: 0.06327360519294892
Validation loss per 100 evaluation steps: 0.05798328291414773
Validation loss per 100 evaluation steps: 0.052577675032816816
Validation loss per 100 evaluation steps: 0.05320250427355572
Validation loss per 100 evaluation steps: 0.0529378443953978
Validation loss per 100 evaluation steps: 0.05024368767995779
Validation loss per 100 evaluation steps: 0.052405190896894334
Validation loss per 100 evaluation steps: 0.05039172644334014
Validation loss per 100 evaluation steps: 0.053030124451324226
Validation loss per 100 evaluation steps: 0.052946254398343905
Validation loss per 100 evaluation steps: 0.05269726613120591
Validation Loss: 0.05241826333197258
Validation Accuracy: 0.9878044353083829


However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library: 

In [ ]:
# from seqeval.metrics import classification_report
from sklearn.metrics import classification_report

print(len(labels))
print(len(predictions))
print(classification_report(labels, predictions))

65787
65787
              precision    recall  f1-score   support

      Device       0.93      0.92      0.93      2072
           O       0.99      0.99      0.99     60651
    Resource       0.90      0.94      0.92      2414
     Service       0.96      0.88      0.92       650

    accuracy                           0.99     65787
   macro avg       0.95      0.93      0.94     65787
weighted avg       0.99      0.99      0.99     65787



#### **Saving the model for future use**

Finally, let's save the vocabulary, model weights and the model's configuration file to a directory, so that they can be re-loaded using the `from_pretrained()` class method.



In [ ]:
# torch.save(model, '/content/drive/MyDrive/ColabNotebooks/NER_model/Architectures/model_BERT_NER_50')

In [16]:
# Model class must be defined somewhere
model = torch.load('/content/drive/MyDrive/ColabNotebooks/NER_model/Architectures/model_BERT_NER_50')
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
# import os

# directory = "./model"

# if not os.path.exists(directory):
#     os.makedirs(directory)

# # save vocabulary of the tokenizer
# tokenizer.save_vocabulary(directory)
# # save the model weights and its configuration file
# model.save_pretrained(directory)
# print('All files saved')
# print('This tutorial is completed')

All files saved
This tutorial is completed


#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences. 
Here, we use the prediction of the **first word piece of every word**. Note that the function we used to train our model (`tokenze_and_preserve_labels`) propagated the label to all subsequent word pieces (so you could for example also perform a majority vote on the predicted labels of all word pieces of a word).

*In other words, the code below does not take into account when predictions of different word pieces that belong to the same word do not match.*

In [17]:
def practical_test(sentence):
  inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")
  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model(ids, mask)
  logits = outputs[0]

  active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  word_level_predictions = []
  for pair in wp_preds:
      if (pair[0].startswith("##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
          # skip prediction
          continue
      else:
          word_level_predictions.append(pair[1])

  # we join tokens, if they are not special ones
  str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
  print(str_rep)
  print(word_level_predictions)
  return [str_rep,word_level_predictions]

In [18]:
userStory="""
As a Doctor, I want to the system to notify me about abnormality in sensor readings.
As a Doctor, I want to store in a database any abnormality in breathing of the patient.
As an Admin, I want to remove doctors and patients details of the MySQL.
As an Admin, I want to my iphone to alert me about some DDOS attacks.
As a Doctor, I want to upload a specific image and video of the patient.
"""




print("\n run 1")
for sentence in userStory.split('.'):
  result=practical_test(sentence)






 run 1
as a doctor , i want to the system to notify me about abnormality in sensor readings
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
as a doctor , i want to store in a database any abnormality in breathing of the patient
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Resource', 'O', 'O', 'O', 'O', 'O', 'O']
as an admin , i want to remove doctors and patients details of the mysql
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
as an admin , i want to my iphone to alert me about some ddos attacks
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Device', 'O', 'O', 'O', 'O', 'O', 'O']
as a doctor , i want to upload a specific image and video of the patient
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Resource', 'O', 'Resource', 'O', 'O']

[]


In [20]:
userStory="""
As a Doctor, I want to the system to notify me about abnormality in sensor readings.
As a Doctor, I want to store in a database any abnormality in breathing of the patient.
As an Admin, I want to remove doctors and patients details of the MySQL.
As an Admin, I want to my iphone to alert me about some DDOS attacks.
As a Doctor, I want to upload a specific image and video of the patient.
"""
r=practical_test(userStory)


as a doctor , i want to the system to notify me about abnormality in sensor readings . as a doctor , i want to store in a database any abnormality in breathing of the patient . as an admin , i want to
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Device', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Resource', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
# wordAndClass=[]

# for sentence in storyLine1.split('.'):
#   result=practical_test(sentence)
#   i=0
#   for pred in result[1]:   
#     if 'Service' == pred:
#       print(pred,result[0].split()[i])
#     if 'Device' == pred:
#       print(pred,result[0].split()[i])
#     if 'Resource' == pred:
#       print(pred,result[0].split()[i])
#     i=1+i

# for word in wordAndClass:
#   storyLine1=storyLine1.replace(word[0], "")


In [ ]:
ll=['O', 'O', 'Device', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Device', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
ll.index(3)

ValueError: ignored

## Legacy

In [ ]:
def prepare_sentence(sentence, tokenizer, maxlen):    
      # step 1: tokenize the sentence
      tokenized_sentence = tokenizer.tokenize(sentence)
      
      # step 2: add special tokens 
      tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] 

      # step 3: truncating/padding
      if (len(tokenized_sentence) > maxlen):
        # truncate
        tokenized_sentence = tokenized_sentence[:maxlen]
      else:
        # pad
        tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]

      # step 4: obtain the attention mask
      attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
      
      # step 5: convert tokens to input ids
      ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)
      
      return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(attn_mask, dtype=torch.long),
            #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
      }